<a href = "https://www.pieriantraining.com"><img src="../PT Centered Purple.png"> </a>

<em style="text-align:center">Copyrighted by Pierian Training</em>

In [1]:
from azure.identity import AzureCliCredential
from azure.mgmt.rdbms.postgresql_flexibleservers import PostgreSQLManagementClient
from azure.mgmt.rdbms.postgresql_flexibleservers.models import Server, Sku, Storage

import psycopg2

from settings import AZURE_SUBSCRIPTION_ID, DEFAULT_LOCATION, DEFAULT_RESOURCE_GROUP

In [2]:
credential = AzureCliCredential()
postgres_client = PostgreSQLManagementClient(credential, AZURE_SUBSCRIPTION_ID)

In [3]:
poller = postgres_client.servers.begin_create(
    DEFAULT_RESOURCE_GROUP,
    'bens-server',
    Server(
        location=DEFAULT_LOCATION,
        administrator_login='benadmin',
        administrator_login_password='testpassword123!',
        sku=Sku(name='Standard_D2s_v3', tier='GeneralPurpose'),
        storage=Storage(storage_size_gb=32),
        version='13'
    )
)
server = poller.result()

In [4]:
server.name

'bens-server'

In [5]:
poller = postgres_client.firewall_rules.begin_create_or_update(
    DEFAULT_RESOURCE_GROUP,
    'bens-server',
    'AllowMyIp',
    {
        'start_ip_address': '72.174.17.20',
        'end_ip_address': '72.174.17.20'
    }
)
firewall_rule = poller.result()

In [6]:
firewall_rule.name

'AllowMyIp'

In [7]:
for server in postgres_client.servers.list():
    print(server.name)

bens-server


In [9]:
poller = postgres_client.databases.begin_create(
    DEFAULT_RESOURCE_GROUP,
    'bens-server',
    'bens-db',
    {}
)
db = poller.result()

In [10]:
db.name

'bens-db'

In [11]:
for db in postgres_client.databases.list_by_server(DEFAULT_RESOURCE_GROUP, 'bens-server'):
    print(db.name)

azure_maintenance
postgres
azure_sys
bens-db


In [26]:
firewall = sql_client.firewall_rules.create_or_update(
    DEFAULT_RESOURCE_GROUP,
    'bens-server',
    'JupyterNotebook',
    {
        'start_ip_address': '72.175.52.98',
        'end_ip_address': '72.175.52.98'
    }
)


In [27]:
HOST_NAME = 'bens-server'
DB_NAME = 'bens-db'
USER_NAME = 'benadmin'
PASSWORD = 'testpassword123!'

In [13]:
connection = psycopg2.connect(
    host='bens-server.postgres.database.azure.com',
    database='bens-db',
    user='benadmin',
    password='testpassword123!'
)

In [14]:
cursor = connection.cursor()

In [16]:
cursor.execute("""
    select table_name, table_schema
    from information_schema.tables
    limit 15;
""")
rows = cursor.fetchall()

In [17]:
rows

[('pg_statistic', 'pg_catalog'),
 ('pg_type', 'pg_catalog'),
 ('pg_foreign_table', 'pg_catalog'),
 ('pg_roles', 'pg_catalog'),
 ('pg_settings', 'pg_catalog'),
 ('pg_hba_file_rules', 'pg_catalog'),
 ('pg_config', 'pg_catalog'),
 ('pg_shmem_allocations', 'pg_catalog'),
 ('pg_prepared_statements', 'pg_catalog'),
 ('pg_stat_progress_vacuum', 'pg_catalog'),
 ('pg_replication_origin_status', 'pg_catalog'),
 ('pg_subscription', 'pg_catalog'),
 ('pg_attribute', 'pg_catalog'),
 ('pg_proc', 'pg_catalog'),
 ('pg_class', 'pg_catalog')]

In [18]:
cursor.close()
connection.close()